::: {.content-visible when-meta="verbose"}
# Initialization
:::

In [ ]:
# General R and plotting 
library(tidyverse)
library(ggplot2)
library(scales)
library(patchwork)
library(cowplot)
library(gridExtra)
library(ggrepel)
library(stringr)
library(VennDiagram)
library(pheatmap)
library(viridis)
library(here)

# Single Cell Analysis Packages
library(scRepertoire)
library(scRepertoire)
library(circlize)
library(scCustomize)
library(SingleR)
library(celldex)
library(UCell)
library(scplotter)

# DEG, pathway enrichment and visualization packages
library(DESeq2)
library(clusterProfiler)
library(DOSE)
library(pathview)
library(org.Mm.eg.db)
library(scRepertoire)
library(enrichplot)
library(msigdbr)
library(gprofiler2)

source(here('scripts/function_template.r'))

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.2     ✔ tibble    3.3.0
✔ lubridate 1.9.4     ✔ tidyr     1.3.1
✔ purrr     1.1.0     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors

Attaching package: ‘scales’


The following object is masked from ‘package:purrr’:

    discard


The following object is masked from ‘package:readr’:

    col_factor



Attaching package: ‘cowplot’


The following object is masked from ‘package:patchwork’:

    align_plots


The following object is masked from ‘package:lubridate’:

    stamp



Attaching package: ‘gridExtra’


The following object is masked from ‘package:dplyr’:

    combine


Loading required package: 

In [2]:
i_am('scripts/Analysis.ipynb')
here()
path <- here()
results_path <- here('results')
dir.create(results_path)
figures_path <- here('results/figures')
dir.create(figures_path)
data_path <- here('data')
dir.create(data_path)

here() starts at /Users/eduardansaldo/Scripps Research Dropbox/Eduard Ansaldo Gine/Constantinides lab/Experiments/Eduard/202507_lung_Abau_Ftul_CT_MAIT_Dom



[1] "/Users/eduardansaldo/Scripps Research Dropbox/Eduard Ansaldo Gine/Constantinides lab/Experiments/Eduard/202507_lung_Abau_Ftul_CT_MAIT_Dom"

:::{.content-visible when-meta='verbose'}
# Preprocessing
:::

:::{.content-visible when-meta='verbose'}
## Read in data
:::

In [ ]:
#options(Seurat.object.assay.version='v3')

#Load the dataset from the cellranger outs
scdata <- Read10X(data.dir = here("cluster_processing/aggr_all/outs/count/filtered_feature_bc_matrix"))

#Initialize the seurat object with the raw (non-normalized data)
seurat <- CreateSeuratObject(counts=scdata$'Gene Expression', min.cells = 3)

# #Add HTO data as a new assay independent from RNA
HTO <- CreateAssayObject(counts = scdata$'Antibody Capture')
seurat[["HTO"]] <- HTO

[1] "/Users/eduardansaldo/Scripps Research Dropbox/Eduard Ansaldo Gine/Constantinides lab/Experiments/Eduard/202507_lung_Abau_Ftul_CT_MAIT_Dom/scripts"

[1] "/Users/eduardansaldo/Scripps Research Dropbox/Eduard Ansaldo Gine/Constantinides lab/Experiments/Eduard/202507_lung_Abau_Ftul_CT_MAIT_Dom/scripts"

here() starts at /Users/eduardansaldo/Scripps Research Dropbox/Eduard Ansaldo Gine/Constantinides lab/Experiments/Eduard/202507_lung_Abau_Ftul_CT_MAIT_Dom



[1] "/Users/eduardansaldo/Scripps Research Dropbox/Eduard Ansaldo Gine/Constantinides lab/Experiments/Eduard/202507_lung_Abau_Ftul_CT_MAIT_Dom"

:::{.content-visible when-meta='processing'}
## QC
:::

In [ ]:
:::{.content-visible when-meta='processing'}

#Quantifying percentage mitochondria
seurat[["percent.mt"]] <- PercentageFeatureSet(seurat, assay="RNA",pattern = "mt-")

VlnPlot(seurat, features = c("nFeature_RNA", "nCount_RNA", "percent.mt"), ncol=3)

p1 <- seurat@meta.data %>% 
  	ggplot(aes(x=nCount_RNA)) + 
  	geom_density() + 
  	scale_x_log10() + 
  	theme_classic() +
  	ylab("Cell density") +
  	geom_vline(xintercept = 1200)

p2 <- seurat@meta.data %>% 
  	ggplot(aes(x=nFeature_RNA)) + 
  	geom_density() + 
  	scale_x_log10() + 
  	theme_classic() +
  	ylab("Cell density") +
  	geom_vline(xintercept = 500)

p3 <- seurat@meta.data %>% 
  	ggplot(aes(x=percent.mt)) + 
  	geom_density() + 
  	scale_x_log10() + 
  	theme_classic() +
  	ylab("Cell density") +
  	geom_vline(xintercept = 5)

grid.arrange(p1, p2, p3)    

# # Visualize feature relationships
# # plot1 <- FeatureScatter(seurat, feature1 = "nCount_RNA", feature2 = "percent.mt")

# # plot2 <- FeatureScatter(seurat, feature1 = "nCount_RNA", feature2 = "nFeature_RNA")
# # plot1+plot2

cell_number <- list(nrow(seurat[[]]))

:::

In [ ]:
:::{.content-visible when-meta='processing'}
# Filter data

cell_number <- list(nrow(seurat[[]]))
seurat_old <- seurat
seurat <- subset(seurat, subset = nFeature_RNA  > 500 & percent.mt < 5 & nCount_RNA > 1200)
cell_number <- append(cell_number, nrow(seurat[[]]))

print("Cell number before and after filterings")
print(cell_number)


:::

:::{.content-visible when-meta='verbose'}
## Data Normalization
:::

In [ ]:
# Normalize RNA data with log normalization
seurat <- NormalizeData(seurat, assay = "RNA",normalization.method = "LogNormalize", scale.factor = 10000)

# Normalize HTO data with CLR
seurat <- NormalizeData(seurat, assay = "HTO",normalization.method = "CLR")

#Implement SCTransform

:::{.content-visible when-meta='processing'}
## Hashtag Demultiplexing
:::

In [ ]:
:::{.content-visible when-meta='processing'}
colnames(seurat@meta.data)
plot1 <- seurat@meta.data %>% 
  	ggplot(aes(x=nCount_HTO)) + 
  	geom_density() + 
  	scale_x_log10() + 
  	theme_classic() +
  	ylab("Cell density") +
  	geom_vline(xintercept = 100)

plot2 <- seurat@meta.data %>% 
  	ggplot(aes(x=nFeature_HTO)) + 
  	geom_density() + 
  	scale_x_log10() + 
  	theme_classic() +
  	ylab("Cell density") +
  	geom_vline(xintercept = 10)

plot1+plot2
:::

In [ ]:
:::{.content-visible when-meta='processing'}

seurat <- HTODemux(seurat, assay = "HTO", positive.quantile = 0.99)

table(seurat$HTO_classification.global)

#tSNE-visualization
seurat_subset <- subset(seurat, idents = "Negative", invert =TRUE)
DefaultAssay(seurat_subset) <- "HTO"
seurat_subset <- ScaleData(seurat_subset, assay = "HTO",  features = rownames(seurat_subset), verbose = FALSE )
seurat_subset <- RunPCA(seurat_subset, assay = "HTO", rownames(seurat_subset), approx = FALSE)
seurat_subset <- RunTSNE(seurat_subset, assay = "HTO", dims = 1:24, perplexity = 100, check_duplicates=FALSE)
DimPlot(seurat_subset)

:::

In [ ]:
:::{.content-visible when-meta='processing'}

print("Number of Singlets")
print()(nrow(seurat@meta.data |> filter("HTO_classification.global" == 'Singlet')))

print("Number of Doublets")
print(nrow(seurat@meta.data |> filter("HTO_classification.global" == 'Doublet')))

print("Number of Negatives")
print(nrow(seurat@meta.data |> filter("HTO_classification.global" == 'Negative')))


# Filtering singlets only
Idents(seurat) <- "HTO_classification.global"
seurat <-subset(seurat, idents = "Singlet")

:::

:::{.content-visible when-meta='verbose'}
## Create Groups
:::

In [ ]:
seurat@meta.data <- seurat@meta.data |>
    mutate(
        Groups = case_when(
            str_detect(hash.ID, 'SPF-iso') ~ 'SPF isotype',
            str_detect(hash.ID, 'SPF-aCD3') ~ 'SPF aCD3',
            str_detect(hash.ID, 'GF-iso') ~ 'GF isotype',
            str_detect(hash.ID, 'GF-aCD3') ~ 'GF aCD3',
            TRUE ~ 'Other'

    ),
        Samples = hash.ID,
        Condition = case_when(
            str_detect(hash.ID, 'SPF') ~ 'SPF',
            str_detect(hash.ID, 'GF') ~ 'GF',
            TRUE ~ 'Other'
        ),
        Treatment = case_when(
            str_detect(hash.ID, 'iso') ~ 'isotype',
            str_detect(hash.ID, 'aCD3') ~ 'aCD3',
        ) 
        )


seurat@meta.data <- seurat@meta.data |>
    mutate(Groups = factor(Groups,levels = c('SPF isotype', 'GF isotype', 'SPF aCD3', 'GF aCD3'))) |>
    mutate(Samples = factor(Samples,levels = c('SPF-iso-1', 
                                                'SPF-iso-2',
                                                'SPF-iso-3',
                                                'SPF-iso-4',
                                                'SPF-aCD3-1',
                                                'SPF-aCD3-2',
                                                'SPF-aCD3-3',
                                                'GF-iso-1',
                                                'GF-iso-2',
                                                'GF-iso-3',
                                                'GF-iso-4',
                                                'GF-aCD3-1',
                                                'GF-aCD3-2',
                                                'GF-aCD3-3',
                                                'GF-aCD3-4'                                                
                                                ))) 
    
#Create groups
colnames(seurat@meta.data)
levels(seurat$Groups)
levels(seurat$Samples)

:::{.content-visible when-meta='processing'}
## Clustering
:::

In [ ]:
:::{.content-visible when-meta='processing'}

# Find and scale variable features
DefaultAssay(seurat) <- "RNA"
seurat <- FindVariableFeatures(seurat, selection.method = "vst", nfeatures = 2000)

top25  <- head(VariableFeatures(seurat), 25)

# plot1 <- VariableFeaturePlot(seurat)
# plot2 <- LabelPoints(plot = plot1, points=top25, repel=TRUE)
# plot1+plot2

#Scaling the Data
all.genes <- rownames(seurat)
seurat <- ScaleData(seurat, features=all.genes)

#Dimensionality reduction
seurat <- RunPCA(seurat, features = VariableFeatures(object=seurat), npcs=200)

#Determining dimensionality of the dataset
ElbowPlot(seurat, ndims = 200)

:::

In [ ]:
# Describe number of dimensions

dimensions  <- 45


In [ ]:
#| output: false
resolutions <- c(0.25, 0.5, 1, 1.25, 1.5, 1.75, 2, 3, 4, 5)
seurat <- FindNeighbors(seurat, dims = 1:dimensions)
seurat <- FindClusters(seurat, resolution = resolutions)
seurat <- RunUMAP(seurat, dims=1:dimensions )

In [ ]:
:::{.content-visible when-meta='processing'}

p <- list()
i <- 1
for (resolution in resolutions ) {
    Idents(seurat) <- paste0('RNA_snn_res.', resolution) 
    p2 <- DimPlot(seurat, reduction = "umap", label = TRUE) + ggtitle(paste0('R ', resolution)) + theme(legend.position = "none")    
    p[[i]] <- p2
    i <- i+1
}

plot <- grid.arrange(grobs = p)
# ggsave('initial_clustering_results_by_resolution.pdf', path = here('result/figures'), plot = plot)

:::

:::{.content-visible when-meta='processing'}
Chosen resolution
:::

In [ ]:
:::{.content-visible when-meta='processing'}

#| echo: true
resolution <- 0.25

:::

:::{.content-visible when-meta='processing'}
## Removing contaminating clusters
:::

:::{.content-visible when-meta='processing'}
### Looking for Outlier Clusters
:::

In [ ]:
:::{.content-visible when-meta='processing'}

Idents(seurat) <- paste0('RNA_snn_res.', resolution)
seurat[['seurat_clusters']]<- Idents(seurat)
DimPlot(seurat, reduction = "umap", label = TRUE) + ggtitle(paste0('R ', resolution))
DimPlot(seurat, reduction = "umap", label = TRUE, split.by = 'Groups', ncol = 3) + ggtitle(paste0('R ', resolution)) + theme(legend.position = "none")

:::

:::{.content-visible when-meta='processing'}
### Automatic cell type annotations
:::

In [ ]:
:::{.content-visible when-meta='processing'}

local_path <- paste0(results_path, 'initial_cell_type_annotations')
unlink(local_path,recursive = T)
dir.create(local_path)

:::

:::{.content-visible when-meta='processing'}
Annotation per Group 
:::

In [ ]:
:::{.content-visible when-meta='processing'}

immgen <- ImmGenData(ensembl = FALSE)
predictions_cluster <- SingleR(test = as.SingleCellExperiment(seurat), assay.type.test = 1, ref = immgen, labels = immgen$label.fine, cluster = seurat$seurat_clusters)

row.names <- rownames(predictions_cluster)

predictions_cluster <- predictions_cluster |>
    as_tibble() |> 
    dplyr::select(c('labels')) 

predictions_cluster$cluster <- row.names

#predictions_cluster <- predictions_cluster |> rename('labels_per_cluster' = 'labels')

annotations <- seurat@meta.data |>
    left_join(predictions_cluster, by = join_by('seurat_clusters' == 'cluster')) |>
    pull(labels)

seurat$labels_per_cluster <- annotations

Idents(seurat) <- 'labels_per_cluster'
DimPlot(seurat, label = T, label.size = 2.5)
ggsave(paste0('UMAP_cluster_SingleR_annotations','.pdf'), path = local_path, width = 8, height = 5)
DimPlot(seurat, label = T, label.size = 2.5, split.by = 'Groups')
ggsave(paste0('UMAP_cluster_SingleR_annotations_by_group','.pdf'), path = local_path, width = 13, height = 5)

:::

:::{.content-visible when-meta='processing'}
Annotations per Sample
:::

In [ ]:
:::{.content-visible when-meta='processing'}

immgen <- ImmGenData(ensembl = FALSE)
predictions_cell_basis <- SingleR(test = as.SingleCellExperiment(seurat), assay.type.test = 1, ref = immgen, labels = immgen$label.main)

predictions_cell_basis <- predictions_cell_basis |>
    as_tibble() |> 
    dplyr::select(c('labels')) 

predictions_cell_basis <- predictions_cell_basis |> rename('labels_per_cell_coarse' = 'labels')

seurat$labels_per_cell_coarse <- predictions_cell_basis |> pull(labels_per_cell_coarse)
    
Idents(seurat) <- 'labels_per_cell_coarse'
DimPlot_scCustom(seurat, label = F)#+ theme(legend.position = 'none')
ggsave(paste0('UMAP_cell_SingleR_annotations_coarse','.pdf'), path = local_path, width = 5, height = 5)
DimPlot_scCustom(seurat, label = F, split.by = 'Groups')#+ theme(legend.position = 'none')
ggsave(paste0('UMAP_cell_SingleR_annotations_coarse_by_group','.pdf'), path = local_path, width = 13, height = 5)

:::

:::{.content-visible when-meta='verbose'}
### Top genes per cluster 
:::

In [ ]:
object_annotations <- 'pre_filtering'
 
Idents(seurat) <- 'seurat_clusters'

seurat.markers <- FindAllMarkers(seurat, only.pos = TRUE, min.pct = 0.25, logfc.threshold = 0.25)

# saveRDS(seurat.markers, file = 'seurat.markers.rds')



#Add gene annotations:
annotations <- read.csv(here("scripts/annotations.csv"))
seurat.markers <- seurat.markers |>
                left_join(y= unique(annotations[,c('gene_name', 'description')]),
                    by = c('gene' = 'gene_name'))

                    

#Top10 markers
seurat.markers %>%
    group_by(cluster) %>%
    arrange(desc(avg_log2FC)) |>
    slice_head(n = 10) -> top10

#Top25 markers
seurat.markers %>%
    group_by(cluster) %>%
    arrange(desc(avg_log2FC)) |>
    slice_head(n = 25) -> top25

#Top100 markers
seurat.markers %>%
    group_by(cluster) %>%
    arrange(desc(avg_log2FC)) |>
    slice_head(n = 100) -> top100

#Top3 markers
seurat.markers %>%
    group_by(cluster) %>%
    arrange(desc(avg_log2FC)) |>
    slice_head(n = 3) -> top3

write.table(top100,file=paste0(path,'top100', '_',object_annotations, ".tsv"), sep="\t",row.names = FALSE)
# write.table(top25,file=paste0(path,'top25',object_annotations, ".tsv"), sep="\t",row.names = FALSE)
write.table(top10,file=paste0(path,'top10', '_',object_annotations, ".tsv"), sep="\t",row.names = FALSE)


# # #Top5 markers
# seurat.markers %>%
#     group_by(cluster) %>%
#     slice_head(n = 5) -> top5
# DoHeatmap(seurat, features = top5$gene, size = 1) + NoLegend() & theme(text=element_text(size=6))
# ggsave(paste0("Heatmap_Top5_per_cluster", object_annotations, ".pdf"), path = path)

# #Top10 markers
# seurat.markers %>%
#      group_by(cluster) %>%
#      slice_head(n = 10) -> top10
#  DoHeatmap(seurat, features = top10$gene, size = 1) + NoLegend() & theme(text=element_text(size=4)) 
#  ggsave(paste0("Heatmap_Top10_per_cluster", object_annotations, ".pdf"), path = path)

gene_list_plot <- top3 |> pull(gene)

pdf(paste0(figures_path, 'Top_3_per_cluster_dotplot_', object_annotations, '.pdf'))
Clustered_DotPlot(seurat, features=gene_list_plot, k=length(unique(seurat$seurat_clusters)), plot_km_elbow = FALSE, cluster_ident=FALSE, ggplot_default_colors = T)

dev.off()


    

:::{.content-visible when-meta='processing'}
### Plotting individual genes of interest for cluster identification
:::

In [ ]:
:::{.content-visible when-meta='processing'}

FeaturePlot_scCustom(seurat, features = c('Cd4', 'Tbx21', 'Vil1', 'Cd19', 'Cd3e', 'Foxp3', 'Rorc', 'Gata3', 'Trbc1', 'Trbc2', 'Trac'))


:::

:::{.content-visible when-meta='processing'}
### Subsetting 
:::

In [ ]:
:::{.content-visible when-meta='processing'}

Idents(seurat) <- 'seurat_clusters'
seurat <- subset(seurat, idents = c('13', '16', '3', '15', '18'), invert = T)
DimPlot(seurat, label = T)

:::

In [ ]:
saveRDS(seurat, here('data', 'scRNAseq_aCD3.rds'))

:::{.content-visible when-meta='processing'}
## Reclustering
:::

In [ ]:
# Normalize RNA data with log normalization
seurat <- NormalizeData(seurat, assay = "RNA",normalization.method = "LogNormalize", scale.factor = 10000)

In [ ]:
:::{.content-visible when-meta='processing'}

# Find and scale variable features
DefaultAssay(seurat) <- "RNA"
seurat <- FindVariableFeatures(seurat, selection.method = "vst", nfeatures = 2000)

top25  <- head(VariableFeatures(seurat), 25)

# plot1 <- VariableFeaturePlot(seurat)
# plot2 <- LabelPoints(plot = plot1, points=top25, repel=TRUE)
# plot1+plot2

#Scaling the Data
all.genes <- rownames(seurat)
seurat <- ScaleData(seurat, features=all.genes)

#Dimensionality reduction
seurat <- RunPCA(seurat, features = VariableFeatures(object=seurat), npcs=200)

#Determining dimensionality of the dataset
ElbowPlot(seurat, ndims = 200)

:::

In [ ]:
# Describe number of dimensions

dimensions  <- 45


In [ ]:
#| output: false
resolutions <- c(0.25, 0.5, 1, 1.25, 1.5, 1.75, 2, 3, 4, 5)
seurat <- FindNeighbors(seurat, dims = 1:dimensions)
seurat <- FindClusters(seurat, resolution = resolutions)
seurat <- RunUMAP(seurat, dims=1:dimensions )

In [ ]:
:::{.content-visible when-meta='processing'}

p <- list()
i <- 1
for (resolution in resolutions ) {
    Idents(seurat) <- paste0('RNA_snn_res.', resolution) 
    p2 <- DimPlot(seurat, reduction = "umap", label = TRUE) + ggtitle(paste0('R ', resolution)) + theme(legend.position = "none")    
    p[[i]] <- p2
    i <- i+1
}

plot <- grid.arrange(grobs = p)
# ggsave('initial_clustering_results_by_resolution.pdf', path = here('result/figures'), plot = plot)

:::

:::{.content-visible when-meta='processing'}
Chosen resolution
:::

In [ ]:
:::{.content-visible when-meta='processing'}

#| echo: true
resolution <- 0.25

:::

# Analysis

## Clustering Results

In [ ]:
Idents(seurat) <- paste0('RNA_snn_res.', resolution)
seurat[['seurat_clusters']]<- Idents(seurat)
DimPlot(seurat, reduction = "umap", label = TRUE) + ggtitle(paste0('R ', resolution)) + theme(legend.position = "none")
ggsave(paste0('UMAP_clusters_R_', resolution, '.pdf'), path = figures_path)
DimPlot(seurat, reduction = "umap", label = TRUE, split.by = 'Groups', ncol = 2) + ggtitle(paste0('R ', resolution)) + theme(legend.position = "none")
ggsave(paste0('UMAP_clusters_by_group_R_', resolution, '.pdf'), path = figures_path, width = 12, height = 5)
DimPlot(seurat, reduction = "umap", label = TRUE, split.by = 'Samples', ncol = 4) + ggtitle(paste0('R ', resolution)) + theme(legend.position = "none")
ggsave(paste0('UMAP_clusters_by_sample_R_', resolution, '.pdf'), path = figures_path)

## Cluster Proportions

In [ ]:
#path <- './'
# Extracting cell counts for bar graphs
cell_counts <- FetchData(seurat,vars=c('seurat_clusters', "Samples")) 
cell_counts <- arrange(cell_counts, Samples)


counts <- cell_counts %>% add_count(Samples, name='total_cell_count_by_sample') #%>%  ungroup() %>% arrange( seurat_clusters , desc(hash.ID) ) #%>% arrange('seurat_clusters')

counts <- counts %>% 
    dplyr::count(seurat_clusters, Samples, total_cell_count_by_sample,name='cluster_count')  |> 
        mutate(frequency_within_sample=cluster_count*100/total_cell_count_by_sample)  |> 
        mutate(Samples = as.character(Samples)) |> 

        arrange( Samples, desc(Samples)) #%>% arrange('seurat_clusters')

new_counts <- counts |> 
    arrange(Samples) |>     
    pivot_wider(id_cols = seurat_clusters, names_from = 'Samples', values_from = frequency_within_sample)
    
write.csv(new_counts,file=paste0(path,"number of cells per cluster per condition.csv"),row.names=F)

colors <- c('deeppink', 'deeppink3', 'hotpink', 'hotpink4', 'cadetblue', 'cadetblue2', 'deepskyblue', 'deepskyblue4')
# Barplot of proportion of cells in each cluster by sample
ggplot(seurat@meta.data) +
    geom_bar(aes(x=Groups, fill=seurat_clusters), position=position_fill()) 
ggsave(paste0(figures_path, 'cells_per_cluster_per_group.pdf'))

# Barplot of proportion of cells in each cluster by sample
ggplot(seurat@meta.data) +
    geom_bar(aes(x=Samples, fill=seurat_clusters), position=position_fill())+ theme(axis.text.x = element_text(angle = 45, vjust = 1))
ggsave(paste0(figures_path, 'cells_per_cluster_per_sample.pdf'))
    

## Top gene per cluster

In [ ]:
object_annotations <- 'global'
 
Idents(seurat) <- 'seurat_clusters'

seurat.markers <- FindAllMarkers(seurat, only.pos = TRUE, min.pct = 0.25, logfc.threshold = 0.25)

# saveRDS(seurat.markers, file = 'seurat.markers.rds')



#Add gene annotations:
annotations <- read.csv(here("scripts/annotations.csv"))
seurat.markers <- seurat.markers |>
                left_join(y= unique(annotations[,c('gene_name', 'description')]),
                    by = c('gene' = 'gene_name'))

                    

#Top10 markers
seurat.markers %>%
    group_by(cluster) %>%
    arrange(desc(avg_log2FC)) |>
    slice_head(n = 10) -> top10

#Top25 markers
seurat.markers %>%
    group_by(cluster) %>%
    arrange(desc(avg_log2FC)) |>
    slice_head(n = 25) -> top25

#Top100 markers
seurat.markers %>%
    group_by(cluster) %>%
    arrange(desc(avg_log2FC)) |>
    slice_head(n = 100) -> top100

#Top3 markers
seurat.markers %>%
    group_by(cluster) %>%
    arrange(desc(avg_log2FC)) |>
    slice_head(n = 3) -> top3

write.table(top100,file=paste0(path,'top100', '_',object_annotations, ".tsv"), sep="\t",row.names = FALSE)
# write.table(top25,file=paste0(path,'top25',object_annotations, ".tsv"), sep="\t",row.names = FALSE)
write.table(top10,file=paste0(path,'top10', '_',object_annotations, ".tsv"), sep="\t",row.names = FALSE)


# # #Top5 markers
# seurat.markers %>%
#     group_by(cluster) %>%
#     slice_head(n = 5) -> top5
# DoHeatmap(seurat, features = top5$gene, size = 1) + NoLegend() & theme(text=element_text(size=6))
# ggsave(paste0("Heatmap_Top5_per_cluster", object_annotations, ".pdf"), path = path)

# #Top10 markers
# seurat.markers %>%
#      group_by(cluster) %>%
#      slice_head(n = 10) -> top10
#  DoHeatmap(seurat, features = top10$gene, size = 1) + NoLegend() & theme(text=element_text(size=4)) 
#  ggsave(paste0("Heatmap_Top10_per_cluster", object_annotations, ".pdf"), path = path)

gene_list_plot <- top3 |> pull(gene)

pdf(paste0(figures_path, 'Top_3_per_cluster_dotplot_', object_annotations, '.pdf'))
Clustered_DotPlot(seurat, features=gene_list_plot, k=length(unique(seurat$seurat_clusters)), plot_km_elbow = FALSE, cluster_ident=FALSE, ggplot_default_colors = T)

dev.off()


    

:::{.content-visible when-meta='processing'}
### Automatic cell type annotations
:::

In [ ]:
:::{.content-visible when-meta='verbose'}

local_path <- paste0(results_path,object_annotations,'cell_type_annotations')
unlink(local_path,recursive = T)
dir.create(local_path)

:::

Annotation per Group 

In [ ]:
:::{.content-visible when-meta='processing'}

immgen <- ImmGenData(ensembl = FALSE)
predictions_cluster <- SingleR(test = as.SingleCellExperiment(seurat), assay.type.test = 1, ref = immgen, labels = immgen$label.fine, cluster = seurat$seurat_clusters)

row.names <- rownames(predictions_cluster)

predictions_cluster <- predictions_cluster |>
    as_tibble() |> 
    dplyr::select(c('labels')) 

predictions_cluster$cluster <- row.names

#predictions_cluster <- predictions_cluster |> rename('labels_per_cluster' = 'labels')

annotations <- seurat@meta.data |>
    left_join(predictions_cluster, by = join_by('seurat_clusters' == 'cluster')) |>
    pull(labels)

seurat$labels_per_cluster <- annotations

Idents(seurat) <- 'labels_per_cluster'
DimPlot(seurat, label = T, label.size = 2.5)
ggsave(paste0('UMAP_cluster_SingleR_annotations','.pdf'), path = local_path, width = 8, height = 5)
DimPlot(seurat, label = T, label.size = 2.5, split.by = 'Groups')
ggsave(paste0('UMAP_cluster_SingleR_annotations_by_group','.pdf'), path = local_path, width = 13, height = 5)

:::

Annotations per Sample

In [ ]:
:::{.content-visible when-meta='processing'}

immgen <- ImmGenData(ensembl = FALSE)
predictions_cell_basis <- SingleR(test = as.SingleCellExperiment(seurat), assay.type.test = 1, ref = immgen, labels = immgen$label.main)

predictions_cell_basis <- predictions_cell_basis |>
    as_tibble() |> 
    dplyr::select(c('labels')) 

predictions_cell_basis <- predictions_cell_basis |> rename('labels_per_cell_coarse' = 'labels')

seurat$labels_per_cell_coarse <- predictions_cell_basis |> pull(labels_per_cell_coarse)
    
Idents(seurat) <- 'labels_per_cell_coarse'
DimPlot_scCustom(seurat, label = F)#+ theme(legend.position = 'none')
ggsave(paste0('UMAP_cell_SingleR_annotations_coarse','.pdf'), path = local_path, width = 5, height = 5)
DimPlot_scCustom(seurat, label = F, split.by = 'Groups')#+ theme(legend.position = 'none')
ggsave(paste0('UMAP_cell_SingleR_annotations_coarse_by_group','.pdf'), path = local_path, width = 13, height = 5)

:::

In [ ]:
#| output: false
immgen <- ImmGenData(ensembl = FALSE)
predictions_cell_basis <- SingleR(test = as.SingleCellExperiment(seurat), assay.type.test = 1, ref = immgen, labels = immgen$label.fine)

predictions_cell_basis <- predictions_cell_basis |>
    as_tibble() |> 
    dplyr::select(c('labels')) 

predictions_cell_basis <- predictions_cell_basis |> rename('labels_per_cell_fine' = 'labels')

seurat$labels_per_cell_fine <- predictions_cell_basis |> pull(labels_per_cell_fine)
    
Idents(seurat) <- 'labels_per_cell_fine'
DimPlot_scCustom(seurat, label = F)#+ theme(legend.position = 'none')
ggsave(paste0('UMAP_cell_SingleR_annotations_fine','.pdf'), path =local_path, width = 17, height = 5)
DimPlot_scCustom(seurat, label = F, split.by = 'Groups')#+ theme(legend.position = 'none')
ggsave(paste0('UMAP_cell_SingleR_annotations_fine_by_group','.pdf'), path =local_path, width = 30, height = 5)

:::{.content-visible when-meta='verbose'}
## Save Object
:::

In [ ]:
saveRDS(seurat, file = 'scRNAseq_.rds')

## Plotting Genes of Interest

## Annotating clusters

In [ ]:
Idents(seurat) <- 'seurat_clusters'
seurat <- RenameIdents(seurat
    , '0' = 'Tbet+ (Steady state)'  
    , '1' = 'Treg'
    , '2' = 'Tbet+ (In vivo activated)'
    , '3' = 'LN-like'
    , '4' = 'Tbet+ (In vivo activated)'
    , '5' = 'iNKT (Steady state)'
    , '6' = 'Th17 (In vivo activated)'
    , '7' = 'Th17 (Steady state)' 
    , '8' = 'iNKT (In vivo activated)'
    , '9' = 'Proliferating Tregs'
    , '10' = 'LN-like'
    , '11' = 'LN-like'
    , '12' = 'Cycling'
    , '13' = 'ND'
    , '14' = 'Treg'
    , '16' = 'Tbet+ (In vivo activated)'
    , '17' = 'Th17 (Steady state)'
    , '18' = 'ISG'
    , '19' = 'LN-like'
)
seurat[['cell_types']] <- Idents(seurat)

In [ ]:
Idents(seurat) <- 'cell_types'
DimPlot(seurat, label=F, ncol = 2) 
ggsave('UMAP_cell_types.pdf', path = figures_path)

DimPlot(seurat, label=F, split.by = 'Groups', ncol = 2)
ggsave('UMAP_cell_types_by_group.pdf', path = figures_path, width = 12, height = 5)

DimPlot(seurat, label=F, split.by = 'Samples', ncol = 2)
ggsave('UMAP_cell_types_by_sample.pdf', path = figures_path, width = 12, height = 5)


:::{.content-visible when-meta='verbose'}
## Saving object
:::

In [ ]:
saveRDS(seurat, file='scRNAseq_.rds')

## Cell frequency per condition

In [ ]:
#path <- './'
# Extracting cell counts for bar graphs
cell_counts <- FetchData(seurat,vars=c('cell_types', "Samples")) 
cell_counts <- arrange(cell_counts, Samples)


counts <- cell_counts %>% add_count(Samples, name='total_cell_count_by_sample') #%>%  ungroup() %>% arrange( cell_types , desc(hash.ID) ) #%>% arrange('cell_types')

counts <- counts %>% 
    dplyr::count(cell_types, Samples, total_cell_count_by_sample,name='cluster_count')  |> 
        mutate(frequency_within_sample=cluster_count*100/total_cell_count_by_sample)  |> 
        mutate(Samples = as.character(Samples)) |> 

        arrange( Samples, desc(Samples)) #%>% arrange('cell_types')

new_counts <- counts |> 
    arrange(Samples) |>     
    pivot_wider(id_cols = cell_types, names_from = 'Samples', values_from = frequency_within_sample)
    
write.csv(new_counts,file=paste0(path,"number of cells per cluster per condition.csv"),row.names=F)

colors <- c('deeppink', 'deeppink3', 'hotpink', 'hotpink4', 'cadetblue', 'cadetblue2', 'deepskyblue', 'deepskyblue4')
# Barplot of proportion of cells in each cluster by sample
ggplot(seurat@meta.data) +
    geom_bar(aes(x=Groups, fill=cell_types), position=position_fill()) 
ggsave(paste0(figures_path, 'cells_per_cluster_per_group.pdf'))

# Barplot of proportion of cells in each cluster by sample
ggplot(seurat@meta.data) +
    geom_bar(aes(x=Samples, fill=cell_types), position=position_fill())+ theme(axis.text.x = element_text(angle = 45, vjust = 1))
ggsave(paste0(figures_path, 'cells_per_cluster_per_sample.pdf'))
    

# Differential gene expression analysis

## Pseudobulk analysis by condition

In [ ]:
## SPF
genes_of_interest <- list('suppressive signature' = c('Il10', 'Ctla4', 'Tigit', 'Havcr2', 'Gzma', 'Lag3', 'Foxp3'))

path <- paste0(results_path, 'DEG_analysis_SPF_aCD3_vs_isotype/')
unlink(path,recursive = T)
dir.create(path)
Idents(seurat) <- 'cell_types'

DEG_counts <- data.frame(matrix(ncol=3, nrow=0))
colnames(DEG_counts) <- c('DEG_count', 'DEG_UP_count', 'DEG_DOWN_count')
rnames <- c('all_cell_types')

scRNAseq_small <- subset(seurat, subset = Condition == 'SPF')
unique(scRNAseq_small$Samples)

DEG_counts <- pseudobulk(scRNAseq_small, comparison='Samples', group1='iso', group2='aCD3', cluster='all_clusters', path=path, label_threshold = 10000, max_overlaps = 25,gene_lists_to_plot = genes_of_interest, FC_threshold = 1, p_value_threshold = 0.01)

for (x in unique(seurat$cell_types_coarse)) {
    scRNAseq1 <- subset(scRNAseq_small, subset = cell_types_coarse == x )
    counts <- pseudobulk(scRNAseq1, comparison='Samples', group1='iso', group2='aCD3', cluster= x , path=path, label_threshold = 10000, max_overlaps = 25,gene_lists_to_plot = genes_of_interest, FC_threshold = 1,p_value_threshold = 0.01)
    DEG_counts <- rbind(DEG_counts, counts)
    rnames <- c(rnames, x)
}